SELECT PROPER FILENAME K VALUE AND MIN_OVERLAP VALUES

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from node2vec import Node2Vec
import logging
import os

# Set up logging to help with debugging and tracking the progress of the program.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def read_sequences_from_file(filename):
    """Read DNA sequences and their associated class labels from a file.
    
    Args:
        filename (str): The path to the file containing sequences and labels, separated by tabs.
    
    Returns:
        list of tuples: A list where each tuple contains a sequence and its corresponding label.
    """
    with open(filename, 'r') as file:
        return [line.strip().split('\t') for line in file if line.strip()]

def chop_into_kmers(sequences, k):
    """Break down each sequence into k-mers of specified length.
    
    Args:
        sequences (list): A list of tuples containing sequences and their labels.
        k (int): The length of each k-mer.
    
    Returns:
        list of tuples: Each tuple contains a k-mer and its associated label.
    """
    return [(seq[i:i+k], label) for seq, label in sequences for i in range(len(seq) - k + 1)]

def find_overlaps(kmers, min_overlap_length):
    """Identify all k-mers that overlap by at least the minimum specified length.
    
    Args:
        kmers (list): A list of k-mers.
        min_overlap_length (int): Minimum length of overlap.
    
    Returns:
        list of tuples: Each tuple represents an overlap between two k-mers, including the overlap size.
    """
    overlaps = []
    for i, (kmer1, _) in enumerate(kmers):
        for j, (kmer2, _) in enumerate(kmers):
            if i != j:
                length = min(len(kmer1), len(kmer2))
                for ol in range(min_overlap_length, length + 1):
                    if kmer1.endswith(kmer2[:ol]):
                        overlaps.append((kmer1, kmer2, ol))
                        break
    return overlaps

def construct_overlap_graph(overlaps):
    """Construct a graph where each node represents a k-mer and edges represent overlaps.
    
    Args:
        overlaps (list): A list of overlaps between k-mers.
    
    Returns:
        networkx.Graph: A graph object with k-mers as nodes connected by their overlaps.
    """
    graph = nx.Graph()
    for kmer1, kmer2, weight in overlaps:
        graph.add_edge(kmer1, kmer2, weight=weight)
    return graph

def plot_graph(graph, file_name):
    """Visualize the overlap graph using Matplotlib.
    
    Args:
        graph (networkx.Graph): The graph to visualize.
        file_name (str): Path to save the visualization.
    """
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(graph, scale=2, k=1/(graph.order()**0.5)*2)
    labels = {node: node[:5] + '...' + node[-5:] for node in graph.nodes()}
    nx.draw(graph, pos, labels=labels, with_labels=True, node_size=50, font_size=8)
    plt.savefig(file_name)
    plt.close()

def node2vec_embedding(graph):
    """Embed graph nodes into a vector space using the Node2Vec algorithm.
    
    Args:
        graph (networkx.Graph): The graph to embed.
    
    Returns:
        gensim.models.Word2Vec: A trained Node2Vec model, or None if the graph is empty.
    """
    if graph.number_of_nodes() == 0:
        logging.warning("Graph is empty. Skipping embedding.")
        return None
    node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)
    return node2vec.fit(window=10, min_count=1, batch_words=4)

def export_graph(graph, export_file_name):
    """Export the graph to a GraphML file format.
    
    Args:
        graph (networkx.Graph): The graph to export.
        export_file_name (str): Path where the GraphML file will be saved.
    """
    nx.write_graphml(graph, export_file_name)

def evaluate_graph(graph):
    """Calculate and log key graph metrics to assess its structure and connectivity.
    
    Args:
        graph (networkx.Graph): The graph to evaluate.
    
    Returns:
        dict: Contains the density, number of connected components, and average clustering coefficient.
    """
    density = nx.density(graph)
    num_connected_components = nx.number_connected_components(graph)
    avg_clustering_coefficient = nx.average_clustering(graph)
    logging.info(f"Graph Density: {density}, Connected Components: {num_connected_components}, Average Clustering Coefficient: {avg_clustering_coefficient}")
    return {'density': density, 'num_components': num_connected_components, 'avg_clustering': avg_clustering_coefficient}

def process_kmers(filename, k, min_overlap_values):
    """Main function to process k-mers, construct graphs, and perform classification based on the Node2Vec embeddings.
    
    Args:
        filename (str): Path to the DNA sequences file.
        k (int): Length of each k-mer.
        min_overlap_values (list): List of minimum overlap lengths to consider.
    
    Returns:
        dict: Contains classification reports and accuracy for each combination of k and minimum overlap.
    """
    base_filename = os.path.splitext(os.path.basename(filename))[0]
    results = {}
    sequences = read_sequences_from_file(filename)
    kmers = chop_into_kmers(sequences, k)
    kmer_dict = {kmer: label for kmer, label in kmers}

    for min_overlap in min_overlap_values:
        if min_overlap > k:
            logging.warning(f"Skipping min_overlap={min_overlap} as it is greater than k={k}")
            continue
        overlaps = find_overlaps(kmers, min_overlap)
        graph = construct_overlap_graph(overlaps)
        if not graph.number_of_edges():
            logging.warning(f"Graph is empty for k={k} and min_overlap={min_overlap}. Skipping.")
            continue

        graph_metrics = evaluate_graph(graph)
        graph_dir = f'graphs/{base_filename}_k{k}_min{min_overlap}'
        os.makedirs(graph_dir, exist_ok=True)
        graph_file_name = f'{graph_dir}/overlap_graph_k{k}_min{min_overlap}.png'
        plot_graph(graph, graph_file_name)
        export_graph(graph, f'{graph_dir}/graph_k{k}_min{min_overlap}.graphml')

        model = node2vec_embedding(graph)
        if model is None:
            continue
        embeddings = model.wv.vectors
        node_ids = model.wv.index_to_key
        embeddings_df = pd.DataFrame(embeddings, index=node_ids)
        
        y = pd.Series({node: kmer_dict[node] for node in node_ids if node in kmer_dict})
        X = embeddings_df
        y = y.reindex(X.index)  # Ensure y is aligned with X's index

        if y.nunique() < 2:
            logging.warning(f"Not enough classes to train SVM for k={k} and min_overlap={min_overlap}. Need at least 2 classes.")
            continue

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        classifier = svm.SVC()
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)

        report = classification_report(y_test, y_pred, output_dict=True)
        accuracy = accuracy_score(y_test, y_pred)
        results[(k, min_overlap)] = {'report': report, 'accuracy': accuracy, 'graph_metrics': graph_metrics}
        logging.info(f"Completed min_overlap={min_overlap} for k={k}")

    return results

filename = 'input_data/hum_med_2.txt'
k_values = [150, 250, 350]
min_overlap_values = [50, 100, 149]
for k in k_values:
    results = process_kmers(filename, k, min_overlap_values)
    logging.info(f"Results for k={k}: {results}")

# Print the results
for key, value in results.items():
    print(f"Results for k={key[0]} and min_overlap={key[1]}:")
    print(f"Accuracy: {value['accuracy']}")
    print("Classification Report:")
    print(value['report'])
    print("Graph Metrics:")
    print(value['graph_metrics']['density'], value['graph_metrics']['num_components'], value['graph_metrics']['avg_clustering'])
